In [1]:
input = sort(readlines("input4")

In [87]:
# parse into a data frame
# id, date, start time, end time, bitmask (00 - 59)
using DataFrames
using DataFramesMeta

mutable struct State
    id::Int
    dt::String
    asleep::Vector{Int}
    laststart::Int
end

function parse_line!(s::String, state::Union{Nothing,State}, table::Vector{State})
    minute = parse(Int, s[16:17])
    if match(r"Guard", s) != nothing 
        id = parse(Int, s[27:30])
        dt = s[2:2+10-1]
        if state != nothing
            push!(table, state)
        end
        st =  State(id, dt, fill(0, 60), 0)  # start new state
        return st
    elseif match(r"falls", s) != nothing
        state.laststart = minute
        return state
    elseif match(r"wakes", s) != nothing
        for i in state.laststart:minute-1
            state.asleep[i+1] = 1
        end
        return state
    else
        error("bad line: $s")
    end
    println("what?")
end

function parse_everything(input)
    df = Vector{State}()
    state = nothing
    for s in input
        state = parse_line!(s, state, df)
    end
    df
end

df = parse_everything(input)

241-element Array{State,1}:
 State(2251, "1518-03-27", [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  1, 1, 1, 1, 1, 1, 1, 0, 0, 0], 11)
 State(3319, "1518-03-27", [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 53)
 State(1777, "1518-03-28", [0, 0, 0, 0, 0, 0, 0, 0, 1, 1  …  1, 1, 1, 1, 1, 0, 0, 0, 0, 0], 20)
 State(3347, "1518-03-30", [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 56)
 State(103, "1518-03-31", [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 1, 1, 1, 0, 0, 0], 54) 
 State(3371, "1518-04-01", [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 1, 1, 1, 0, 0, 0, 0, 0, 0], 51)
 State(1777, "1518-04-02", [0, 0, 0, 0, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 8) 
 State(1777, "1518-04-02", [0, 0, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2) 
 State(1777, "1518-04-04", [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  1, 1, 1, 1, 1, 0, 0, 0, 0, 0], 50)
 State(2389, "1518-04-04", [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 0], 0) 
 State(2251,

In [92]:
guard_stats = DataFrame(
    id = [x.id for x in df],
    minutes = [sum(x.asleep) for x in df],
    asleep = [x.asleep for x in df]
)
guard_stats[1:5,:]  #debug

,id,minutes,asleep
,Int64,Int64,Array…
1,2251,46,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]"
2,3319,18,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,1777,37,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]"
4,3347,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
5,103,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]"


In [108]:
# who sleeps the most across all days?
function sleepiest(guard_stats)
    top = @linq guard_stats |>
      by(:id, minutes = sum(:minutes)) |>
      orderby(-:minutes) 
    (id = top[1,:id], minutes = top[1, :minutes])
end
sleepiest(guard_stats)  #debug

(id = 1777, minutes = 407)

In [111]:
function sleepiest_minute(id, guard_stats)
    st = @linq guard_stats |> where(:id .== id)
    mat = vcat([d' for d in st.asleep]...)
    r = DataFrame(minute=0:59, elapsed=[sum(mat[:, c]) for c in 1:60])
    worst = @linq r |> orderby(-:elapsed)
    worst[1, :minute]
end
sleepiest_minute(1777, guard_stats)  #debug

48

In [112]:
# part 1
let id = sleepiest(guard_stats).id
    id * sleepiest_minute(id, guard_stats)
end

85296

In [119]:
# part 2
function stats_by_guard(id)
    A = vcat([st.asleep' for st in filter(x -> x.id == id, df)]...)
    x = findmax([sum(A[:,i]) for i in 1:60])
    (id = id, days_asleep = x[1], minutes = x[2] - 1, days = size(A, 1))
end

function sleepiest_minute(guard_stats)
    df = DataFrame([stats_by_guard(id) for id in unique(guard_stats.id)])
    df = @linq df |> 
      orderby(-:days_asleep) |>
      transform(answer = :id .* :minutes)
    (id = df[1, :id], minutes = df[1, :minutes], answer = df[1, :answer])
end

sleepiest_minute(guard_stats)

(id = 1889, minutes = 31, answer = 58559)